In [ ]:
library

In [69]:
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import string
from nltk import pos_tag
import warnings
warnings.filterwarnings("ignore")
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn import tree

Importing Training Data

In [44]:
df_train = pd.read_csv('D:\\CDC\\train.csv')

In [70]:
df_train = df_train[['text', 'airline_sentiment']]

In [71]:
training_data = df_train.values

Spliiting the Tweet text into words using NLTK

In [72]:
tweets_train = []
for i in range(len(training_data)):
    tweets_train.append([word_tokenize(training_data[i][0]), training_data[i][1]])  

Cleaning the Words using WordNetLemmatizer available in NLTK

In [73]:
stops = set(stopwords.words('english'))
punctuations = list(string.punctuation)
stops.update(punctuations)

In [74]:
from nltk.corpus import wordnet
def get_simple_pos(tag):
    
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [75]:
lemmatizer = WordNetLemmatizer()
def clean_tweets(words):
    output_words = []
    for w in words:
        if w.isalpha():
            if w.lower() not in stops:
                pos = pos_tag([w])
                clean_word = lemmatizer.lemmatize(w, pos = get_simple_pos(pos[0][1]))
                output_words.append(clean_word.lower())
    return output_words

In [76]:
import nltk

# Download the missing resource
nltk.download('averaged_perceptron_tagger')
import nltk

# Download the missing resource
nltk.download('wordnet')

# Your existing code
for i in range(len(tweets_train)):
    tweets_train[i] = (clean_tweets(tweets_train[i][0]), tweets_train[i][1])


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\ABHIJIT\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ABHIJIT\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [77]:
y_train = []
tweets = []
for tweet, sentiment in tweets_train:
    tweets.append(" ".join(tweet))
    y_train.append(sentiment)

Using Count Vectorizer to get the X Train

In [78]:
count_vec = CountVectorizer(max_features=2000) # Tried using n grams but the accuracy was decreasing
x_train_features = count_vec.fit_transform(tweets)

Prepaing Testing Data

In [79]:
df_test = pd.read_csv('D:\\CDC\\test.csv')

In [80]:
testing_data = np.array(df_test['text'])

In [81]:
tweets_test = []
for t in testing_data:
    t = clean_tweets(word_tokenize(t))
    tweets_test.append(" ".join(t))

In [82]:
x_test_features = count_vec.transform(tweets_test)

Performing Classification

Support Vector Machine

In [83]:
svc = SVC()
svc.fit(x_train_features, y_train)

SVC()

In [84]:
y_pred_svm = svc.predict(x_test_features)

In [85]:
df = pd.DataFrame(y_pred_svm)
df.to_csv('predictions_svm.csv', index = False, header = False)

In [86]:
print("True Labels Shape:", true_sentiments.shape)
print("Predicted Labels Shape:", predicted_sentiments.shape)

True Labels Shape: (10980,)
Predicted Labels Shape: (3660,)


Random Forest

In [26]:
rf = RandomForestClassifier()
rf.fit(x_train_features, y_train)

RandomForestClassifier()

In [27]:
y_pred_rf = rf.predict(x_test_features)

In [28]:
df = pd.DataFrame(y_pred_rf)
df.to_csv('predictions_rf.csv', index = False, header = False)

Multinomial Naive Bayes

In [29]:
mnv = MultinomialNB(alpha = 1)
mnv.fit(x_train_features, y_train)

MultinomialNB(alpha=1)

In [30]:
y_pred_mnv = mnv.predict(x_test_features)

In [31]:
df = pd.DataFrame(y_pred_mnv)
df.to_csv('predictions_mnv.csv', index = False, header = False)

Descision Tree

In [32]:
dt = tree.DecisionTreeClassifier()
dt.fit(x_train_features, y_train)

DecisionTreeClassifier()

In [33]:
y_pred_dt = dt.predict(x_test_features)

In [34]:
df = pd.DataFrame(y_pred_dt)
df.to_csv('predictions_dt.csv', index = False, header = False)